# Fitbit Food Data

This notebook walks through how to pull sleep, food & calories out data using the Fitbit API.

In [14]:
%load_ext autoreload
%autoreload 2

In [15]:
import sys
sys.path.append('../src/')
import plotutils
import fitbitqueries as fbq

In [16]:
import fitbit
import os
import getpass
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Connect to fitbit API

In [2]:
# os.environ['client_id'] = '22BVDT'
# os.environ['client_secret'] = '1c4962ccbb977e158d95c7ef1676d690'

In [3]:
os.environ['client_id'] = '22BC6L'
os.environ['client_secret'] = 'e8348b3b97bba4db4fb142928cedfc31'

In [4]:
!/Users/hasannagib/opt/anaconda3/envs/fitbit/bin/python ../src/generate_tokens.py $client_id $client_secret

[30/Jul/2020:11:05:59] ENGINE Listening for SIGTERM.
[30/Jul/2020:11:05:59] ENGINE Listening for SIGHUP.
[30/Jul/2020:11:05:59] ENGINE Listening for SIGUSR1.
[30/Jul/2020:11:05:59] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[30/Jul/2020:11:05:59] ENGINE Started monitor thread 'Autoreloader'.
[30/Jul/2020:11:06:00] ENGINE Serving on http://127.0.0.1:8080
[30/Jul/2020:11:06:00] ENGINE Bus STARTED
127.0.0.1 - - [30/Jul/2020:11:06:01] "GET /?code=21a2da0bda7dd7f9b12629a12354b649ed869dd8&state=xrSZtobRPoPg41aCpWiWgeATWxneEC HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"
[30/Jul/2020:11:06:02] ENGINE Bus STOPPING
[30/Jul/2020:11:06:02] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[30/Jul/2020:11:06:02] ENGINE Stopped thread 'Autoreloader'.
[30/Jul/2020:11:06:02] ENGINE Bus STOPPED
[30/Jul/2020:11:06:02] ENGI

In [5]:
with open('../data/access_token.txt','r') as f:
    os.environ['fitbit_access_token'] = f.read()

with open('../data/refresh_token.txt','r') as f:
    os.environ['fitbit_refresh_token'] = f.read()
    
auth_client = fitbit.Fitbit(
    os.environ['client_id'], 
    os.environ['client_secret'],
    os.environ['fitbit_access_token'],
    os.environ['fitbit_refresh_token']
)

## Query fitbit data

In [6]:
date_list = pd.date_range('2020-01-17', '2020-04-17')

food_logs = [auth_client.foods_log(date) for date in date_list]
cal_out = [(date, auth_client.activities(date)['summary']['caloriesOut']) for date in date_list]
macros_data = [[food_log['summary'], date] for food_log, date in zip(food_logs, date_list)]
food_data = []

for food_log in food_logs:
    for item in food_log['foods']:
        food_data.append([item['logDate'], item['loggedFood']['name'], item['loggedFood']['calories'], item['loggedFood']['mealTypeId']])
    

In [7]:
df_calsout = pd.DataFrame(cal_out, columns=['log_date', 'caloriesOut']).set_index('log_date')

In [8]:
# Prepare foods dataframes
df_foods = pd.DataFrame(food_data, columns=['date', 'name', 'calories', 'meal'])

In [9]:
df_top_foods = df_foods.groupby('name').sum().sort_values('calories', ascending=False)[['calories']]
df_top_foods['% of total calories'] = 100*(df_top_foods['calories']/df_foods.sum()['calories'])
df_top_foods['% calories (cumulative)'] = df_top_foods['% of total calories'].cumsum()

In [10]:
df_macros = pd.DataFrame([data[0] for data in macros_data])
df_macros['log_date'] = [data[1] for data in macros_data]
df_macros['carbs(%)'] = 100*(df_macros['carbs']*4)/((df_macros['carbs']*4) + (df_macros['fat']*9) + (df_macros['protein']*4) )
df_macros['fat(%)'] = 100*(df_macros['fat']*9)/((df_macros['carbs']*4) + (df_macros['fat']*9) + (df_macros['protein']*4) )
df_macros['protein(%)'] = 100*(df_macros['protein']*4)/((df_macros['carbs']*4) + (df_macros['fat']*9) + (df_macros['protein']*4) )
df_macros = df_macros.set_index('log_date')
df_macros = df_macros.join(df_calsout)

In [24]:
df_macros['calsIn_over_calsOut'] = df_macros['calories']/df_macros['caloriesOut']
df_macros['carbs_target'] = 450
df_macros['protein_target'] = 250
df_macros['fiber_target'] = 36
df_macros['fat_target'] = 150
df_macros.head()

,calories,carbs,fat,fiber,protein,sodium,water,carbs(%),fat(%),protein(%),caloriesOut,calsIn_over_calsOut,carbs_target,protein_target,fiber_target,fat_target
log_date,,,,,,,,,,,,,,,,
2020-01-17,4338,638.46,131.50,67.49,185.40,3803.52,0,57.018848,26.423663,16.557489,3347,1.296086,450,250,36,150
2020-01-18,3031,325.61,135.34,52.79,154.31,1549.87,0,41.508857,38.819660,19.671483,3434,0.882644,450,250,36,150
2020-01-19,2531,287.23,74.42,27.04,174.17,4146.56,0,45.675802,26.627388,27.696809,2248,1.125890,450,250,36,150
2020-01-20,3783,586.08,94.72,65.04,169.51,6697.92,0,60.501079,22.000392,17.498529,3489,1.084265,450,250,36,150
2020-01-21,3585,460.03,127.29,52.18,171.55,7898.84,0,50.113156,31.199124,18.687720,3826,0.937010,450,250,36,150


In [35]:
def plot_macro(df, macros):
    plot_macros = plotutils.plot_ts(
        df, 
        ys=macros + [f'{macro}_target' for macro in macros], 
        styles=['o-']*len(macros) + ['-']*len(macros),
        date_col='log_date',
        title='Macros (grams)',
        legend_location='top_left'
    )
    show(plot_macros)

In [38]:
plot_macro(
    df_macros.reset_index().groupby(pd.Grouper(key="log_date", freq="3D")).mean(), 
    ['protein', 'carbs', 'fat']
)

Assuming carbs and protein provide `4 cals` per `gram` and fat provides `9 cals` per `gram`, here are the macros percentages

## What do I actually eat...
Let's inspect the food logs since last week of November. Where do most of my calories come from?

In [39]:
print(f"Food by cals:\n{df_foods['date'].min()} to {df_foods['date'].max()}")
df_top_foods.head(50)

Food by cals:
2020-01-17 to 2020-04-17


,calories,% of total calories,% calories (cumulative)
name,,,
Bananas,48668,14.719644,14.719644
CanadianProtein,18711,5.659145,20.378789
Mass Gainer,14628,4.424241,24.803029
Chicken Breast,14210,4.297817,29.100846
Quinoa,11932,3.608835,32.709681
Pork Loin (Country-Style Ribs),11512,3.481806,36.191487
All Natural Peanut Butter,10764,3.255573,39.447061
12 Grain Bagels,9050,2.737174,42.184234
Banana,8761,2.649766,44.834000


In [40]:
df_food_name_mapper = df_top_foods.reset_index()[['name']]
df_food_name_mapper['std_name'] = None
df_food_name_mapper.to_csv('../data/food_name_mapper.csv', index=None)

In [41]:
plotutils.top_food_plot(df_top_foods.head(50))

# Daily food log view

In [23]:
df_foods.tail(50)

,date,name,calories,meal
1058,2020-04-12,"Asparagus (Drained, Cooked, Boiled)",11,2
1059,2020-04-12,Olive Oil,476,2
1060,2020-04-12,Chicken Thigh (Skin Not Eaten),380,2
1061,2020-04-12,Chicken Leg (Skin Not Eaten),359,2
1062,2020-04-12,Egg White,105,2
1063,2020-04-12,Mass Gainer,718,4
1064,2020-04-12,Bananas,726,4
1065,2020-04-13,Olive Oil,476,3
1066,2020-04-13,Chicken Breast,732,3
1067,2020-04-13,Mixed Vegetables (Frozen),91,3
